In [ ]:
# Import dependencies

In [ ]:
# Read in data from csv file

In [ ]:
# Convert to dataframe

In [ ]:
# Clean and transform data
# Drop null values
# Drop unnecessary columns

In [ ]:
# Transform words into vectors using NLP
    # Tokenize descriptions to split sentences into word components
    # Drop stop words
    # Lemmitization
# Separate data in target and features variables
# Preprocess data
# Divide into train and test data

In [ ]:
# Input vector data into ML algorithm
# Apply K-Means Clustering
# Generate scatter plot of results

In [ ]:
# Input vector data into ML algorithm
# Apply 2nd Model (TBD)
# Generate scatter plot of results

In [ ]:
# Classify using Random Forest

In [ ]:
# Print accuracy score and confusion matrix